In [38]:
import pandas as pd
import datetime

In [39]:
DATA_URL = 'http://www.ik2ane.it/pontixls.xls'

In [40]:
FT3D = True

In [41]:
# Number of channels to export Max 900
CHANNELS = 800

In [42]:
# Given list of tones
tones = [67.0, 69.3, 71.9, 74.4, 77.0, 79.7, 82.5, 85.4, 88.5, 91.5, 94.8, 97.4, 100.0, 103.5, 107.2, 110.9, 114.8, 118.8, 123.0, 127.3, 131.8, 136.5, 141.3, 146.2, 151.4, 156.7, 159.8, 162.2, 165.5, 167.9, 171.3, 173.8, 177.3, 179.9, 183.5, 186.2, 189.9, 192.8, 196.6, 199.5, 203.5, 206.5, 210.7, 218.1, 225.7, 229.1, 233.6, 241.8, 250.3, 254.1]

# Add a space and "Hz" at the end of each float number
TONES = [f"{tone} Hz" for tone in tones]


In [43]:
country_mapping = {
    "Svizzera": "xCH",
    "Austria": "xAT",
    "Croazia": "xHR",
    "Francia": "xFR",
    "Germania": "xDE",
    "Slovenia": "xSI",
    "Montenegro": "xME"
}

In [44]:
# Set a constant for the QTH locator
QTH_LOCATOR = "JN34VI"

# JN34VI = Mondovi
# JN45KX = Caslano

In [45]:
df = pd.read_excel(DATA_URL)
        # Use the rename function with a lambda function to replace brackets and convert to lowercase

In [46]:
# df.to_csv('data.csv', index=False)

In [47]:
if FT3D:
    # # Drop rows where 'freq' column is greater than 1000000
    df = df[pd.to_numeric(df['(F)req'], errors='coerce') <= 1000000]
else:
    # For the FTM400
    # Convert the '(F)req' column to numeric and drop rows where values are outside the range [140000, 1000000]
    df = df[(pd.to_numeric(df['(F)req'], errors='coerce') <= 1000000) & (pd.to_numeric(df['(F)req'], errors='coerce') >= 140000)]


In [48]:
# Sorting the dataframe by the distance to my favorite locators"

import math


def valid_maidenhead(grid_square):
    """
    Check if a Maidenhead grid square is valid (i.e., it's a string and has a length of 4 or 6).
    """
    return isinstance(grid_square, str) and (len(grid_square) == 4 or len(grid_square) == 6)


def maidenhead_to_latlon(grid_square):
    """
    Convert a Maidenhead grid square (like FN31pr) to latitude and longitude.
    This function assumes a 4 or 6 character grid square.
    """
    if len(grid_square) < 4:
        return None

    # Get the fields (first two characters)
    field_lon = ord(grid_square[0].lower()) - ord('a')
    field_lat = ord(grid_square[1].lower()) - ord('a')

    # Get the squares (next two characters)
    square_lon = int(grid_square[2])
    square_lat = int(grid_square[3])

    # Calculate the longitude and latitude in degrees
    lon = -180 + 20 * field_lon + 2 * square_lon
    lat = -90 + 10 * field_lat + square_lat

    # If we have a 6 character grid square, get the subsquares
    if len(grid_square) == 6:
        subsquare_lon = ord(grid_square[4].lower()) - ord('a')
        subsquare_lat = ord(grid_square[5].lower()) - ord('a')
        lon += 5/60 * subsquare_lon  # 5 minutes of arc per subsquare
        lat += 2.5/60 * subsquare_lat  # 2.5 minutes of arc per subsquare

    return lat, lon

qth_lat, qth_lon = maidenhead_to_latlon(QTH_LOCATOR)

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points on a sphere given their longitudes and latitudes.
    """
    R = 6371.0  # Radius of the Earth in kilometers

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) ** 2 +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance


def distance_to_qth(grid_square):
    """
    Calculate the distance from a given Maidenhead grid square to the QTH locator.
    """
    if valid_maidenhead(grid_square):
        lat, lon = maidenhead_to_latlon(grid_square)
        return haversine_distance(qth_lat, qth_lon, lat, lon)
    else:
        return float('inf')


# Calculate the distances to the QTH and sort by the shortest distance
df['distance'] = df['QRA (L)oc'].apply(distance_to_qth)

# Sort dataframe by distance
df.sort_values(by="distance", inplace=True)

df = df.reset_index(drop=True)

In [49]:
df.rename(columns=lambda x: x.replace('(', '').replace(')', '').lower(), inplace=True)

# df = df.dropna(subset=['req'])
df = df.drop(['agg.', 'km', 'gradi', 'ordkey', 'jn45ol'], axis=1)
df.dropna(subset=['nome'], inplace=True)

In [50]:
# # Function to version the duplicates in the "nome" column by adding prefix
# def version_duplicates_prefix_explicit(df, column_name):
#     # Create a copy of the series to avoid modifying the original dataframe in-place
#     series = df[column_name].copy()
    
#     counts = series.value_counts()
#     duplicates = counts[counts > 1].index.tolist()
    
#     for duplicate in duplicates:
#         idxs = series[series == duplicate].index
#         for i, idx in enumerate(idxs):
#             versioned_name = f"_{i+1}" + series.loc[idx]
            
#             # Respect the 12 character limit by trimming from the end
#             versioned_name = versioned_name[-12:]
#             series.at[idx] = versioned_name
    
#     # Update the original dataframe with the modified series
#     df[column_name] = series
#     return df

# # Apply the updated versioning function
# df = version_duplicates_prefix_explicit(df, "nome")


In [51]:
# # Extract the rows containing the duplicates for display
# duplicate_rows_updated = df[df["nome"].duplicated(keep=False)].sort_values(by="nome")

# duplicate_rows_updated[["nome"]]

In [52]:
# Remove the whitespace from the column "Name"
df['nome'] = df['nome'].str.strip()

# Rename "HotSpot" as "HS"
df['nome'] = df['nome'].replace('HotSpot', 'HS')

# Replace "Hotspot" with "HS" 
df['nome'] = df['nome'].replace('Hotspot', 'HS')

# Replace blanks in Name with "_"
df['nome'] = df['nome'].str.replace(r'\s+', '_', regex=True)

In [53]:
# Maps foreign regions to their country code
df['prov.'] = df['regione'].str.capitalize().map(country_mapping).fillna(df['prov.'])

# Remove content within brackets and the brackets themselves from "località"
df["localita'"] = df["localita'"].str.replace(r'\(.*?\)', '', regex=True).str.replace(" ", "").str.replace("'", "").str.strip()

# Calculate the truncation length for "località" for each row
truncation_lengths = 16 - df['prov.'].str.len() - df['nome'].str.len() - 2

# Replace non-finite values with 0 before converting to integers
truncation_lengths = truncation_lengths.fillna(0).astype(int)

# Truncate the "località" values based on the calculated space
df["truncated_localita'"] = df.apply(lambda row: row["localita'"][:truncation_lengths[row.name]], axis=1)

# Construct the "new_column" again
df['ID'] = df['prov.'] + "_" + df["truncated_localita'"] + "_" + df['nome']

In [54]:
# Rename 'freq' column to 'rx_freq'
df.rename(columns={'freq': 'rx_freq'}, inplace=True)

In [55]:
# Convert frequencies from kHz to MHz
df['rx_freq'] = df['rx_freq'] / 1000

In [56]:
# Define a function to handle the shift
def apply_shift(freq, shift):
    # Check if shift is NaN or a single value without a unit
    if pd.isnull(shift) or len(shift.split()) < 2:
        return freq
    
    # Extract the numeric part and the unit (kHz or MHz) from the shift
    shift_value, shift_unit = shift.split()
    
    # Remove commas from the shift value
    shift_value = shift_value.replace(',', '')
    
    # Handle the '+' sign explicitly
    if shift_value[0] == '+':
        shift_value = shift_value[1:]
    
    # Convert shift value to float
    shift_value = float(shift_value)
    
    # If shift unit is kHz, convert it to kHz (since freq is in kHz)
    if shift_unit == 'kHz':
        shift_value /= 1000
    
    # Return the frequency after applying the shift
    return freq + shift_value


In [57]:
# Apply the shift to the 'rx_freq' column and create a new column 'tx_freq'
df['tx_freq'] = df.apply(lambda row: apply_shift(row['rx_freq'], str(row['shift'])), axis=1)

In [58]:
# Make sure "shift" is handled as a string
df['shift'] = df['shift'].astype(str)

## Moving the list of channels to the radio format

In [59]:
template_df = pd.read_csv('template.csv')

In [60]:
# Initialize a new dataframe with the same columns as the template
export_df = pd.DataFrame(columns=template_df.columns)

In [61]:
# Update the function to handle the shift conversion
def convert_shift(shift):
    # Check if shift is "nan" or "0"
    if shift == "nan" or shift == "0":
        return "", "Simplex"
    
    # Split the shift into parts
    shift_parts = shift.split()
    
    # If shift contains only one part
    if len(shift_parts) == 1:
        # Check if it's a numeric part
        try:
            # Try converting it to float
            shift_value = float(shift_parts[0])
            # If successful, return it with 'Offset Direction' set as "Simplex"
            return shift_value, "Simplex"
        except ValueError:
            # If not a numeric part, return as is
            return shift_parts[0], "Simplex"
    
    # Extract the numeric part and the unit (kHz or MHz) from the shift
    shift_value, shift_unit = shift_parts
    
    # Remove commas from the shift value
    shift_value = shift_value.replace(',', '')
    
    # Handle the '+' or '-' sign explicitly
    if shift_value[0] == '+':
        shift_direction = "Plus"
        shift_value = shift_value[1:]
    elif shift_value[0] == '-':
        shift_direction = "Minus"
        shift_value = shift_value[1:]
    else:
        shift_direction = "Plus"
    
    # Convert shift value to float
    shift_value = float(shift_value)
    
    # If shift unit is MHz, convert it to kHz (since freq is in kHz)
    if shift_unit == 'kHz':
        shift_value /= 1000
    
    # Return the frequency after applying the shift
    return f'{shift_value} MHz', shift_direction



# Apply the shift conversion to the 'shift' column and create new columns 'Offset Frequency' and 'Offset Direction'
export_df['Offset Frequency'], export_df['Offset Direction'] = zip(*df['shift'].map(convert_shift))

# Map the columns in our dataframe to the corresponding columns in the template
export_df['Receive Frequency'] = df['rx_freq']
export_df['Transmit Frequency'] = df['tx_freq']
export_df['Name'] = df['ID']

In [62]:
# Define a function to handle the mapping of 'tono' to 'Tone Mode' and 'CTCSS'
def map_tono(tono):
    if pd.isnull(tono) or tono == "":
        return "None", ""
    else:
        return "Tone", str(tono) + " Hz"


# Define a function to handle the mapping of 'grp' to 'Operating Mode'
def map_grp_to_mode(grp):
    if pd.isnull(grp) or grp == "":
        return "FM"
    elif grp == "C4FM":
        return "DN"
    elif grp in ["DMR", "DS-DMR", "DS", "TDMA", "WL"]:
        return "drop"
    else:
        return "FM"

# Apply the mapping to the 'grp' column and create a new column 'Operating Mode'
export_df['Operating Mode'] = df['grp'].map(map_grp_to_mode)

# Drop the rows where 'Operating Mode' is "drop"
export_df = export_df[export_df['Operating Mode'] != "drop"]


In [63]:
# Define a function to handle the mapping of 'grp' to 'Operating Mode'
def map_grp(grp):
    if grp in ["DMR", "DS-DMR", "DS", "TDMA", "WL"]:
        return "drop"
    else:
        return grp

# Apply the mapping to the 'grp' column in 'df' and create a new column 'grp_mapped'
df['grp_mapped'] = df['grp'].map(map_grp)

# Drop the rows where 'grp_mapped' is "drop" in 'df'
df = df[df['grp_mapped'] != "drop"]

# Drop the 'grp_mapped' column as it's no longer needed
df.drop('grp_mapped', axis=1)

# Apply the mapping to the 'tono' column and create new columns 'Tone Mode' and 'CTCSS'
export_df['Tone Mode'], export_df['CTCSS'] = zip(*df['tono'].map(map_tono))


In [64]:
# Cleaining up the CTCSS column and only take one value from TONES list

def find_closest_tone(ctcss):
    # If the value is NaN or empty, return it as is
    if pd.isna(ctcss):
        return "empty"
    
    # Convert the string to a number, stripping away any non-numeric characters
    try:
        number = float(''.join(filter(str.isdigit, ctcss)))/10  # Dividing by 10 because we are filtering digits and not considering decimal points
    except ValueError:
        return ""
    
    # Compute the absolute difference between this number and each tone in the list
    differences = [abs(number - tone) for tone in tones]
    
    # If the smallest difference is greater than our threshold (1.0), return "empty"
    if min(differences) > 1.0:
        return ""
    
    # Otherwise, return the closest tone from tones_with_units
    closest_tone_value = tones[differences.index(min(differences))]
    return f"{closest_tone_value} Hz"

# Apply the function to the 'CTCSS' column
export_df['CTCSS'] = export_df['CTCSS'].apply(find_closest_tone)

In [65]:
# Set static values in the export_df
export_df["DCS"] = "23"
export_df["DCS Polarity"] = "RN-TN"
export_df["Tx Power"] = "High (5 W)"
export_df["Skip"] = "Off"
export_df["Step"] = "Auto"
export_df["User CTCSS"] = "1600"
export_df["Attenuator"] = "Off"
export_df["Clock Shift"] = "Off"
export_df["Half Dev"] = "Off"

# Set columns 'Bank_1' to 'Bank_24' in the DataFrame to "Off"
for i in range(1, 25):
    export_df[f'Bank_{i}'] = "Off"

export_df["Comment"] = "Python script by IZ1VCX"
export_df["User CTCSS"] = "300 Hz"
export_df["S-Meter Squelch"] = "Off"
export_df["Bell"] = "Off"
export_df["AMS"] = "On"

# Create a column with incremental integers
export_df.iloc[:, -3] = range(1, len(df) + 1)
export_df["RX DGID"] = "0"
export_df["TX DGID"] = "0"


In [66]:
# Set the channel number
export_df['Channel Number'] = range(1, len(export_df) + 1)

# Replace "0.6 MHz" with "600 kHz" in the 'shift' column
export_df['Offset Frequency'] = export_df['Offset Frequency'].replace('0.6 MHz', '600 kHz')

## Add to the beginning fo the dataframe a set of own channels from a list named "private channels"

In [67]:
# private_channels = pd.read_csv("FT3D 09082023_2.csv", nrows=76)
# private_channels.to_csv("FT3D_pvt_channels.csv", index=False)

private_channels = pd.read_csv("FT3D_pvt_channels.csv")
private_channels['Comment'] = "PVT Channels"


In [68]:
# Let's now add the private channels to the export_df
# Concatenate the new data on top of the existing dataframe
my_channel_list = pd.concat([private_channels, export_df], ignore_index=True)

In [69]:
# Resetting the index
my_channel_list.reset_index(drop=True, inplace=True)

In [70]:
# Update the 'Channel Number' column with the new incremental order
my_channel_list['Channel Number'] = my_channel_list.index + 1

In [71]:
# Update the third last column with the new incremental order
my_channel_list.iloc[:, -3] = my_channel_list.index + 1

In [72]:
# Slice and save the dataframe to retain only the first "CHANNELS" rows
my_channel_list.iloc[:CHANNELS].to_csv(f'FT3d_ch_export_{datetime.datetime.now()}.csv', index=False)

In [73]:
my_channel_list

,Channel Number,Receive Frequency,Transmit Frequency,Offset Frequency,Offset Direction,Operating Mode,Name,Tone Mode,CTCSS,DCS,...,Bank_23,Bank_24,Comment,User CTCSS,S-Meter Squelch,Bell,AMS,,RX DGID,TX DGID
0,1,400.0500,400.0500,NaN,Simplex,FM,PAR1LS,NaN,100.0 Hz,23,...,Off,Off,PVT Channels,1500 Hz,Off,Off,Off,1,0,0
1,2,142.2000,142.2000,NaN,Simplex,FM,PAR2LP,NaN,100.0 Hz,23,...,Off,Off,PVT Channels,300 Hz,Off,Off,On,2,0,0
2,3,141.3500,141.3500,NaN,Simplex,FM,PAR3LP,NaN,100.0 Hz,23,...,Off,Off,PVT Channels,300 Hz,Off,Off,On,3,0,0
3,4,146.7800,146.7800,NaN,Simplex,FM,PAR4LI,NaN,100.0 Hz,23,...,Off,Off,PVT Channels,300 Hz,Off,Off,On,4,0,0
4,5,147.8900,147.8900,NaN,Simplex,FM,PAR5MA,NaN,100.0 Hz,23,...,Off,Off,PVT Channels,300 Hz,Off,Off,On,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109,1110,431.6000,433.2000,1.6 MHz,Plus,DN,SR_M.Aquilo_RU31,None,,23,...,Off,Off,Python script by IZ1VCX,300 Hz,Off,Off,On,1110,0,0
1110,1111,145.7375,145.1375,600 kHz,Minus,FM,SR_M.Aquilon_R5a,Tone,77.0 Hz,23,...,Off,Off,Python script by IZ1VCX,300 Hz,Off,Off,On,1111,0,0
1111,1112,430.1250,431.7250,1.6 MHz,Plus,FM,SR_Avola_RU5,Tone,77.0 Hz,23,...,Off,Off,Python script by IZ1VCX,300 Hz,Off,Off,On,1112,0,0
1112,1113,433.5750,433.5750,,Simplex,FM,SR_ColleTirone_T,Tone,123.0 Hz,23,...,Off,Off,Python script by IZ1VCX,300 Hz,Off,Off,On,1113,0,0
